# Computational Finance

## Lab Assignment 2

This notebook makes use of a python 3 environment by default.

Authors:
- Kevin de Vries
- Jedda Boyle
- Krishnakanth Sasi

Student numbers:
- 10579869
- 11398221
- 11391952

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Keeps results reproducible
np.random.seed(23456)

# Part 1: Basic Option Valuation

In [2]:
def European(S0,K,r,sigma,T,N,Z,call):
    """
    Calculates the payoff of European option simulations
    """
    
    S_T = S0 * np.exp((r - sigma**2 / 2)*T + sigma*Z*np.sqrt(T))
    batch = np.maximum(0, S_T - K) if call else np.maximum(0,K - S_T)
    return batch

def MC_option_values(S0,K,r,sigma,T,N,M=100000,N_sim=30,
                     call=False,payoff=European,path=False,antithetic=True,stats=True):
    """
    Calculates a number of option values using a Monte Carlo method. 
    The option to use antithetic variables is on by default.
    
    The value is determined as the mean of a 
    batch of payoffs discounted with the interest rate.
    
    Returns the mean of the calculated options and its standard error
    
    call:
        Determines if call or put option
    payoff:
        Function which calculates the payoff of the option style
    path:
        Determines if the full path of the stock price has to be simulated.
    antithetic:
        Determines if to apply variance reduction using antithetic variables
    stats:
        Determines if the mean and standard error of the option values are returned
    """
    if path:
        Z = np.random.normal(0,1,(M,N_sim,N))
    else:
        Z = np.random.normal(0,1,(M,N_sim))
    
    batch = payoff(S0,K,r,sigma,T,N,Z,call)
    values = np.exp(-r*T) * np.mean(batch,axis=0)
    
    if antithetic:
        batch = payoff(S0,K,r,sigma,T,N,-Z,call)
        values_min = np.exp(-r*T) * np.mean(batch,axis=0)
        values = (values + values_min) / 2
    
    if stats:
        mean,std = data_stats(values)
        return values,mean,std
    else:
        return values
    
def data_stats(data):
    mean = np.mean(data)
    var = np.sum((data-mean)**2) / (data.size - 1)
    std_err = np.sqrt(var/data.size)
    return mean,std_err

In [3]:
# Calculate European option value using Batch Monte Carlo
S0, K = 100, 99
r, sigma = 0.06, 0.2
T, N = 1, 1
M, N_sim = 10000,30

# Option values and stats using mean payoffs
values,mean,std = MC_option_values(S0,K,r,sigma,T,N,M,N_sim,antithetic=False)

print("Mean: %f" % mean)
print("Standard error: %f" % std)

# Option values and stats using mean payoffs with antithetic variables
values,mean,std = MC_option_values(S0,K,r,sigma,T,N,M,N_sim)

print("Mean: %f" % mean)
print("Standard error: %f" % std)

Mean: 4.789625
Standard error: 0.013258
Mean: 4.773392
Standard error: 0.008631


# Part 2: Estimation of Sensitivities in MC

In [4]:
# Calculate European option value using Batch Monte Carlo
S0, K = 100, 99
r, sigma = 0.06, 0.2
T, M = 1,100000
call = False

N_sim = 30
eps = 0.001

# Different seeds
unbumped = MC_option_values(S0,K,r,sigma,T,N,M,N_sim,call)
bumped = MC_option_values(S0+eps,K,r,sigma,T,N,M,N_sim,call)

# Calculate delta from estimate means
deltas = (bumped[0]-unbumped[0]) / eps

mu_delta,std_delta = data_stats(deltas)

print("Different seeds:\n")
print("Mean: %f" % unbumped[1])
print("Standard error: %f\n" % unbumped[2])
print("Hedge parameter estimate: %f" % mu_delta)
print("Hedge parameter standard error: %f\n" % std_delta)

# Same seed
seed = 12

np.random.seed(seed)
unbumped = MC_option_values(S0,K,r,sigma,T,N,M,N_sim,call)
np.random.seed(seed)
bumped = MC_option_values(S0+eps,K,r,sigma,T,N,M,N_sim,call)

# Calculate delta from estimate means
deltas = (bumped[0]-unbumped[0]) / eps

mu_delta,std_delta = data_stats(deltas)

print("Same seed:\n")
print("Mean: %f" % unbumped[1])
print("Standard error: %f\n" % unbumped[2])
print("Hedge parameter estimate: %f" % mu_delta)
print("Hedge parameter standard error: %f" % std_delta)

Different seeds:

Mean: 4.777819
Standard error: 0.002442

Hedge parameter estimate: 2.441588
Hedge parameter standard error: 3.556196

Same seed:

Mean: 4.780265
Standard error: 0.002532

Hedge parameter estimate: -0.326319
Hedge parameter standard error: 0.000098


# Part 3: Variance Reduction

In [5]:
def geometric_Asian(S0,K,r,sigma,T,N,Z,call):
    """
    Calculates the payoff of geometric Asian option simulations
    """
    
    mu = (r-0.5*sigma**2) * (N+1) * T / (2*N)
    std = sigma * np.sqrt((N+1) * (2*N+1) * T / (6*N**2))
    
    S_G = S0*np.exp(mu + std*Z)
    batch = np.maximum(0, S_G - K) if call else np.maximum(0,K - S_G)
    return batch

def arithmetic_Asian(S0,K,r,sigma,T,N,Z,call):
    """
    Calculates the payoff of arithmetic Asian option simulations
    """
    
    dt = T / N
    
    mu = r*dt
    std = sigma*np.sqrt(dt)
    
    M, N_sim, _ = Z.shape
    batch = np.zeros((M,N_sim))
    
    S = np.zeros(N+1)
    S[0] = S0
    for m in range(M):
        for n in range(N_sim):
            for i in range(1,N+1):
                S[i] = S[i-1] + S[i-1]*(mu + std*Z[m,n,i-1])
            S_A = np.mean(S[1:])
            batch[m,n] = max(0,S_A - K) if call else max(0,K - S_A)
    
    return batch

def black_scholes_geometric_asian(S0,K,r,sigma,T,N,call=False):
    mu = (r-0.5*sigma**2) * (N+1) * T / (2*N)
    std = sigma * np.sqrt((N+1) * (2*N+1) * T / (6*N**2))
    
    d2 = (np.log(S0/K) + mu) / std
    d1 = d2 + std
    
    discount = np.exp(mu+0.5*std**2-r*T)
    if call:
        price = S0*discount*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        delta = discount*norm.cdf(d1)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S0*discount*norm.cdf(-d1)
        delta = -discount*norm.cdf(-d1)
    
    return price,delta

In [6]:
# Calculate Asian option value using Batch Monte Carlo
S0, K = 100, 99
r, sigma = 0.06, 0.2
T, N = 1, 1000
M, N_sim = 1000000, 30

eps = 0.001

seed = 12

for call in [True,False]:
    np.random.seed(seed)
    unbumped = MC_option_values(S0,K,r,sigma,T,N,M,N_sim,call,payoff=geometric_Asian)
    np.random.seed(seed)
    bumped = MC_option_values(S0+eps,K,r,sigma,T,N,M,N_sim,call,payoff=geometric_Asian)

    # Calculate delta from estimate means
    deltas = (bumped[0]-unbumped[0]) / eps

    mu_delta,std_delta = data_stats(deltas)

    print("Asian geometric %s\n" % ("call" if call else "put"))
    
    print("Mean: %f" % unbumped[1])
    print("Standard error: %f\n" % unbumped[2])
    print("Hedge parameter estimate: %f" % mu_delta)
    print("Hedge parameter standard error: %f\n" % std_delta)

    price,delta = black_scholes_geometric_asian(S0,K,r,sigma,T,N,call)

    print("Black-Scholes price: %f" % price)
    print("Black-Scholes delta: %f" % delta)
    
    if call:
        print("")

Asian geometric call

Mean: 6.340474
Standard error: 0.000647

Hedge parameter estimate: 0.625298
Hedge parameter standard error: 0.000028

Black-Scholes price: 6.340224
Black-Scholes delta: 0.625284

Asian geometric put

Mean: 2.850569
Standard error: 0.000491

Hedge parameter estimate: -0.341948
Hedge parameter standard error: 0.000028

Black-Scholes price: 2.850401
Black-Scholes delta: -0.341961
